1) raw dataset
2) keywords based extraction
3) train/test split
4) sample a specific number of samples (high cardinality)
5) paste the instruction

In [1]:
from huggingface_hub import logout, notebook_login
# logout()
notebook_login()

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [2]:
## dataset load

from datasets import load_dataset

# qa datasets
medqa = load_dataset("GBaker/MedQA-USMLE-4-options")
medmcqa = load_dataset("openlifescienceai/medmcqa")
pubmedqa = load_dataset("bigbio/pubmed_qa", trust_remote_code=True)
# mfc = load_dataset("veggiebird/medical-flashcards")

# nli datasets
bionli = load_dataset("clinicalnlplab/BioNLI_test")

# information extraction datasets
ddi = load_dataset("YufeiHFUT/DDI2013")
chemdner =load_dataset("kjappelbaum/chemnlp-chemdner")
# medmentions = load_dataset("bigbio/medmentions", trust_remote_code=True) train-108, test-37
# biorel = load_dataset("DFKI-SLT/BioRel")
# phee = load_dataset("sarus-tech/phee")

# generation datasets
healthcaremagic = load_dataset("lighteval/med_dialog", "healthcaremagic")
icliniq = load_dataset("lighteval/med_dialog", "icliniq")
pubmed = load_dataset("akemiH/NoteChat")
# pubmed = load_dataset("Gaborandi/diabetes_mellitus_type2_pubmed_abstracts")
# pubmed = load_dataset("ccdv/pubmed-summarization")

Repo card metadata block was not found. Setting CardData to empty.


In [3]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# 데이터셋을 train과 test로 80:20 비율로 분할
split_dataset = pubmed["train"].train_test_split(test_size=0.2, seed=42)

# test 데이터를 다시 50:50으로 나눠 test와 validation 생성
test_valid = split_dataset["test"].train_test_split(test_size=0.5, seed=42)

# 최종 DatasetDict 구성
pubmed = DatasetDict({
    "train": split_dataset["train"],
    "test": test_valid["test"],
    "validation": test_valid["train"]
})


pubmed

DatasetDict({
    train: Dataset({
        features: ['data', 'conversation'],
        num_rows: 165600
    })
    test: Dataset({
        features: ['data', 'conversation'],
        num_rows: 20701
    })
    validation: Dataset({
        features: ['data', 'conversation'],
        num_rows: 20700
    })
})

In [4]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

tmp = chemdner['train'].to_pandas()
tmp = tmp.drop(columns=["split"])

train_df, temp_df = train_test_split(tmp, test_size=0.2, shuffle=True, random_state=42)
validation_df, test_df = train_test_split(temp_df, test_size=0.5, shuffle=True, random_state=42)

train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)
test_dataset = Dataset.from_pandas(test_df)

chemdner = DatasetDict({
    "train": train_dataset,
    "validation": validation_dataset,
    "test" : test_dataset,
})

chemdner

DatasetDict({
    train: Dataset({
        features: ['entities', 'text', '__index_level_0__'],
        num_rows: 15552
    })
    validation: Dataset({
        features: ['entities', 'text', '__index_level_0__'],
        num_rows: 1944
    })
    test: Dataset({
        features: ['entities', 'text', '__index_level_0__'],
        num_rows: 1944
    })
})

In [5]:
from datasets import concatenate_datasets


# meddialog = {
#     "train": concatenate_datasets([icliniq['train']]),
#     "validation": concatenate_datasets([icliniq['validation']]),
#     "test": concatenate_datasets([icliniq['test']]),
# }

meddialog = {
    "train": concatenate_datasets([healthcaremagic['train'], icliniq['train']]),
    "validation": concatenate_datasets([healthcaremagic['validation'], icliniq['validation']]),
    "test": concatenate_datasets([healthcaremagic['test'], icliniq['test']]),
}

meddialog

{'train': Dataset({
     features: ['tgt', 'src', 'id'],
     num_rows: 205973
 }),
 'validation': Dataset({
     features: ['tgt', 'src', 'id'],
     num_rows: 25746
 }),
 'test': Dataset({
     features: ['tgt', 'src', 'id'],
     num_rows: 25750
 })}

In [6]:
print("medqa")
print(medqa)
print('------------------------------------------------------------------------------'* 2) 
print("medmcqa")
print(medmcqa)
print('------------------------------------------------------------------------------'* 2) 
print("pubmedqa")
print(pubmedqa)
print('------------------------------------------------------------------------------'* 2) 
print("bionli")
print(bionli)
print('------------------------------------------------------------------------------'* 2) 
print("DDI")
print(ddi)
print('------------------------------------------------------------------------------'* 2)
print("chemdner")
print(chemdner)
print('------------------------------------------------------------------------------'* 2)
print("meddialog")
print(meddialog)
print('------------------------------------------------------------------------------'* 2) 
print("pubmed")
print(pubmed)

medqa
DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx', 'metamap_phrases'],
        num_rows: 10178
    })
    test: Dataset({
        features: ['question', 'answer', 'options', 'meta_info', 'answer_idx', 'metamap_phrases'],
        num_rows: 1273
    })
})
------------------------------------------------------------------------------------------------------------------------------------------------------------
medmcqa
DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 182822
    })
    test: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_name'],
        num_rows: 6150
    })
    validation: Dataset({
        features: ['id', 'question', 'opa', 'opb', 'opc', 'opd', 'cop', 'choice_type', 'exp', 'subject_name', 'topic_na

In [9]:
import pandas as pd
import os
import json
import ast

keywords = [keyword.lower() for keyword in [
    "Diabetes", "Diabetic", 
    "Diabetes Mellitus", "Diabetes Mellitus, Experimental", "Diabetes Mellitus, Type 1",
    "Wolfram Syndrome", "Diabetes Mellitus, Type 2", "Diabetes Mellitus, Lipoatrophic",
    "Diabetes, Gestational", "Donohue Syndrome", "Latent Autoimmune Diabetes in Adults",
    "Prediabetic State", "Diabetes Complications", "Diabesity", "Diabetic Angiopathies",
    "Diabetic Cardiomyopathies", "Diabetic Coma", "Diabetic Ketoacidosis",
    "Diabetic Nephropathies", "Diabetic Neuropathies", "Fetal Macrosomia"
]]

def contains_keywords(text, keywords):
    if not text or not isinstance(text, str): 
        return False
    text_lower = text.lower() 
    return any(keyword.lower() in text_lower for keyword in keywords)

def process_dataset_split(dataset_name, split_name, dataset, text_columns, output_file, keywords=["diabetes", "insulin"]):
    rows = []
    for i in range(len(dataset)):
        row = dataset[i]

        for col in text_columns:
            if col in row:
                text = row[col]

            elif col == "abstract" and "passages" in row:
                if isinstance(row["passages"], list):
                    text = " ".join([p["text"] for p in row["passages"] if p.get("type") == "abstract"])
                else:
                    text = ""
            else:
                text = ""

            if not text:
                continue

            is_related = 1 if contains_keywords(text, keywords) else 0

            rows.append({
                "dataset": dataset_name,
                "split_data": split_name,
                "features": row,  
                "input": text,
                "output": is_related
            })

    if rows:  
        df = pd.DataFrame(rows)
        df.to_csv(output_file, mode="a", header=not os.path.exists(output_file), index=False)
        print(f"{dataset_name} ({split_name}): {len(rows)} rows processed and appended to {output_file}")
    else:
        print(f"{dataset_name} ({split_name}): No rows processed.")

datasets = {
    "medqa": {"splits": medqa, "columns": ["question", "answer"]},
    "medmcqa": {"splits": medmcqa, "columns": ["question"]},
    "pubmedqa": {"splits": pubmedqa, "columns": ["QUESTION", "CONTEXTS", "LONG_ANSWER"]},
    "bionli": {"splits": bionli, "columns": ["query", "answer"]},
    "ddi": {"splits": ddi, "columns": ["text"]},
    "chemdner": {"splits": chemdner, "columns": ['text']},
    "meddialog": {"splits": meddialog, "columns": ["tgt", "src"]},
    "pubmed": {"splits": pubmed, "columns": ["conversation", "data"]},
}


output_file = os.path.join("all_data_filtered_by_keywords.csv")

for dataset_name, details in datasets.items():
    splits = details["splits"]
    text_columns = details["columns"] 

    for split_name, split_data in splits.items():
        process_dataset_split(dataset_name, split_name, split_data, text_columns, output_file, keywords=keywords)


medqa (train): 20356 rows processed and appended to all_data_filtered_by_keywords.csv
medqa (test): 2546 rows processed and appended to all_data_filtered_by_keywords.csv
medmcqa (train): 182822 rows processed and appended to all_data_filtered_by_keywords.csv
medmcqa (test): 6150 rows processed and appended to all_data_filtered_by_keywords.csv
medmcqa (validation): 4183 rows processed and appended to all_data_filtered_by_keywords.csv
pubmedqa (train): 600000 rows processed and appended to all_data_filtered_by_keywords.csv
pubmedqa (validation): 33807 rows processed and appended to all_data_filtered_by_keywords.csv
bionli (train): 11088 rows processed and appended to all_data_filtered_by_keywords.csv
bionli (validation): 25612 rows processed and appended to all_data_filtered_by_keywords.csv
bionli (test): 12616 rows processed and appended to all_data_filtered_by_keywords.csv
ddi (train): 18779 rows processed and appended to all_data_filtered_by_keywords.csv
ddi (test): 5761 rows processe

In [1]:
import pandas as pd

all_df = pd.read_csv("/data/jaesung/llm_for_diabetes/src/data/data1_diabetes/all_data_filtered_by_keywords.csv")

In [2]:
# data_filtered_by_keyword
diabetes = all_df[all_df['output'] == 1]

In [3]:
diabetes.head(2)

,dataset,split_data,features,input,output
30,medqa,train,{'question': 'A 68-year-old man presents to th...,A 68-year-old man presents to the emergency de...,1
34,medqa,train,{'question': 'A 68-year-old man comes to the p...,A 68-year-old man comes to the physician becau...,1


In [ ]:
diabetes.groupby(['dataset', 'split_data'])['output'].agg('count')

dataset    split_data
bionli     test            224
           train           221
           validation      537
chemdner   test             57
           train           522
           validation       71
ddi        test             61
           train           289
           valid            45
meddialog  test           1152
           train          9229
           validation     1064
medmcqa    test             33
           train          1375
           validation       34
medqa      test            124
           train          1069
pubmed     test           4074
           train         30832
           validation     3868
pubmedqa   train         12152
           validation      680
Name: output, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

def re_train_test_split_unique(dataframe, train_ratio=0.8):
    new_rows = []

    grouped = dataframe.groupby('dataset')

    for dataset_name, group in grouped:
        if dataset_name not in ['medqa', 'medmcqa', 'pubmedqa', 'bionli', 'chemdner', 'ddi', 'meddialog', 'pubmed']:
            continue

        # 중복 제거
        group = group.drop_duplicates(subset='features')

        train_data, test_data = train_test_split(
            group,
            test_size=1-train_ratio,
            random_state=42,
        )

        train_data = train_data.copy()
        train_data['split_data'] = 'train'

        test_data = test_data.copy()
        test_data['split_data'] = 'test'

        new_rows.append(train_data)
        new_rows.append(test_data)

    result_df = pd.concat(new_rows, ignore_index=True)
    return result_df

diabetes2 = re_train_test_split_unique(diabetes, train_ratio=0.9)

In [6]:
diabetes2.groupby(['dataset', 'split_data'])['output'].agg('count')

dataset    split_data
bionli     test             76
           train           683
chemdner   test             65
           train           585
ddi        test             40
           train           355
meddialog  test            999
           train          8990
medmcqa    test            145
           train          1297
medqa      test            119
           train          1065
pubmed     test           1703
           train         15325
pubmedqa   test            811
           train          7297
Name: output, dtype: int64

In [7]:
import ast
import pandas as pd

def add_input_to_for_sampling(dataframe):
    dataframe = dataframe.copy()
    
    # INPUT 값을 추출하여 새로운 컬럼에 추가
    def extract_for_sampling(row):
        if row['dataset'] == 'medqa':
            features = ast.literal_eval(row['features'])
            input_text = features.get('question', None)
            answer = features.get('answer', None)
            return {'input': input_text, 'output': answer}
        elif row['dataset'] == 'medmcqa':
            features = ast.literal_eval(row['features'])
            input_text = features.get('question', None)
            answer = ", ".join([
                features.get('opa', ''),
                features.get('opb', ''),
                features.get('opc', ''),
                features.get('opd', ''),
            ]).strip()
            return {'input': input_text, 'output': answer}
        elif row['dataset'] == 'pubmedqa':
            features = ast.literal_eval(row['features'])
            question = features.get('QUESTION', None)
            answer = features.get('LONG_ANSWER', None)
            return {'input': question, 'output': answer}
        elif row['dataset'] == 'bionli':
            features = ast.literal_eval(row['features'])
            query = features.get('query', '') 
            input_text = query.split("INPUT: ")[1].split("[HYP]")[0].strip() if "INPUT: " in query else None
            answer = features.get('answer', None)
            return {'input': input_text, 'output': answer}
        elif row['dataset'] == 'chemdner':
            features = ast.literal_eval(row['features'])
            input_text = features.get('text', None)
            answer = features.get('entities', None)
            if isinstance(answer, list) and len(answer) > 0:
                answer = answer[0]
            else:
                answer = ''
            return {'input': input_text, 'output': answer}
        elif row['dataset'] == 'ddi':
            features = ast.literal_eval(row['features'])
            input_text = features.get('conversations', [])[0].get('value', None).split("INPUT:")[1].split("OUTPUT:")[0].strip()
            answer = features.get('conversations', [])[1].get('value', None)
            return {'input': input_text, 'output': answer}
        elif row['dataset'] == 'meddialog':
            features = ast.literal_eval(row['features'])
            input_text = features.get('src', None).split("Doctor:")[0].replace("Patient:", "").strip()
            answer = features.get('src', None).split("Doctor:")[1].strip()
            return {'input': input_text, 'output': answer}
        elif row['dataset'] == 'pubmed':
            features = ast.literal_eval(row['features'])
            input_text = features.get('conversation', None)
            abstract = features.get('data', None)
            return {'input': input_text, 'output': abstract}

    dataframe['for_sampling'] = dataframe.apply(extract_for_sampling, axis=1)

    # chemdner 의 "" 비율 줄이기
    chemdner_df = dataframe[dataframe['dataset'] == 'chemdner'].copy()
    empty_outputs = chemdner_df[chemdner_df['for_sampling'].apply(lambda x: x['output'] == '')]
    non_empty_outputs = chemdner_df[chemdner_df['for_sampling'].apply(lambda x: x['output'] != '')]
    target_empty_size = int(len(non_empty_outputs) * 0.05 / 0.95) 
    sampled_empty_outputs = empty_outputs.sample(n=min(target_empty_size, len(empty_outputs)), random_state=42)
    balanced_chemdner_df = pd.concat([non_empty_outputs, sampled_empty_outputs])
    other_datasets = dataframe[dataframe['dataset'] != 'chemdner']
    final_dataframe = pd.concat([other_datasets, balanced_chemdner_df])
    
    return final_dataframe

diabetes3 = add_input_to_for_sampling(diabetes2)

In [8]:
diabetes3.head(2)

,dataset,split_data,features,input,output,for_sampling
0,bionli,train,"{'id': 'BioNLI28408138', 'query': ""\nTASK: Ple...",\nTASK: Please classify the relationship betwe...,1,"{'input': '[PRE] Since 1875, controversy has e..."
1,bionli,train,"{'id': 'BioNLI19419299', 'query': '\nTASK: Ple...",\nTASK: Please classify the relationship betwe...,1,{'input': '[PRE] TNF-alpha is a major etiologi...


In [9]:
import os
import pandas as pd
import numpy as np
import time
from sentence_transformers import SentenceTransformer
import faiss
from tqdm import tqdm
import random
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def mmr(query_embedding, doc_embeddings, diversity, top_n):
    faiss.normalize_L2(doc_embeddings)
    faiss.normalize_L2(query_embedding.reshape(1, -1))

    selected_indices = []
    candidate_indices = list(range(len(doc_embeddings)))

    for _ in range(top_n):
        if not candidate_indices:
            break

        if len(selected_indices) == 0:
            selected_idx = candidate_indices[np.argmax(np.dot(doc_embeddings[candidate_indices], query_embedding.T))]
        else:
            selected_embeddings = doc_embeddings[selected_indices]
            similarity_to_selected = np.dot(doc_embeddings[candidate_indices], selected_embeddings.T)
            diversity_scores = np.max(similarity_to_selected, axis=1)
            relevance_scores = np.dot(doc_embeddings[candidate_indices], query_embedding.T).flatten()
            mmr_scores = (1 - diversity) * relevance_scores - diversity * diversity_scores
            selected_idx = candidate_indices[np.argmax(mmr_scores)]

        selected_indices.append(selected_idx)
        candidate_indices.remove(selected_idx)

    return selected_indices

def mmr_sampling(dataframe, sampling_dict, embedding_model='all-MiniLM-L6-v2', batch_size=64, diversity=0.7, seed=42):
    set_seed(seed)

    print("Loading SentenceTransformer model on GPU...")
    model = SentenceTransformer(embedding_model, device="cuda:0")

    sampled_rows = []

    for dataset_name, sample_count in tqdm(sampling_dict.items(), desc="Processing datasets"):
        subset = dataframe[(dataframe['dataset'] == dataset_name) & (dataframe['split_data'] == 'train')].copy()
        num_rows = len(subset)
        if num_rows == 0:
            print(f"No data found for dataset: {dataset_name}")
            continue

        print(f"Generating embeddings for dataset: {dataset_name} ({num_rows} rows)")
        start_time = time.time()

        embeddings = model.encode(
            subset['for_sampling'].astype(str).tolist(),
            batch_size=batch_size,
            convert_to_numpy=True,
            show_progress_bar=True
        )

        elapsed_time = time.time() - start_time
        print(f"Embedding generation took {elapsed_time:.2f} seconds for {num_rows} rows")

        query_embedding = np.mean(embeddings, axis=0)

        print(f"Applying MMR for dataset: {dataset_name}")
        selected_indices = mmr(query_embedding, embeddings, diversity, sample_count)
        sampled_subset = subset.iloc[selected_indices]
        sampled_rows.append(sampled_subset)

    return pd.concat(sampled_rows, ignore_index=True)

# 샘플링 설정
sampling_config = {
    'medqa': 500,
    'medmcqa': 500,
    'pubmedqa': 500, # 1000
    'bionli': 500, # 1500
    'chemdner': 500,
    'ddi': 300,
    'pubmed': 500,
    'meddialog': 500, 
}

set_seed(42)

# 샘플링 실행
sampled_train = mmr_sampling(diabetes3, sampling_config, batch_size=4096, diversity=0.7, seed=42)

# 결과 확인
print(sampled_train.head())
print(f"총 샘플링된 행 수: {len(sampled_train)}")

# 결과 저장
sampled_train.to_csv("final_combined_train_sample.csv")


/data/jaesung/anaconda3/envs/faiss_gpu/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading SentenceTransformer model on GPU...


Processing datasets:   0%|          | 0/8 [00:00<?, ?it/s]

Generating embeddings for dataset: medqa (1065 rows)


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.46s/it]


...

Generating embeddings for dataset: meddialog (8990 rows)


Batches: 100%|██████████| 3/3 [00:06<00:00,  2.08s/it]


Embedding generation took 6.84 seconds for 8990 rows
Applying MMR for dataset: meddialog


Processing datasets: 100%|██████████| 8/8 [00:55<00:00,  6.94s/it]


  dataset split_data                                           features  \
0   medqa      train  {'question': 'A 52-year-old man presents to th...   
1   medqa      train  {'question': 'A 72-year-old woman with type 2 ...   
2   medqa      train  {'question': "A pediatrician is investigating ...   
3   medqa      train  {'question': 'You are conducting a systematic ...   
4   medqa      train  {'question': 'A 50-year-old woman with a histo...   

                                               input  output  \
0  A 52-year-old man presents to the emergency de...       1   
1  A 72-year-old woman with type 2 diabetes melli...       1   
2  A pediatrician is investigating determinants o...       1   
3  You are conducting a systematic review on the ...       1   
4  A 50-year-old woman with a history of schizoph...       1   

                                        for_sampling  
0  {'input': 'A 52-year-old man presents to the e...  
1  {'input': 'A 72-year-old woman with type 2 dia...  

In [10]:
import os
import pandas as pd
import numpy as np
import time
from sentence_transformers import SentenceTransformer
import faiss
from tqdm import tqdm
import random
import torch

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def mmr(query_embedding, doc_embeddings, diversity, top_n):
    faiss.normalize_L2(doc_embeddings)
    faiss.normalize_L2(query_embedding.reshape(1, -1))

    selected_indices = []
    candidate_indices = list(range(len(doc_embeddings)))

    for _ in range(top_n):
        if not candidate_indices:
            break

        if len(selected_indices) == 0:
            selected_idx = candidate_indices[np.argmax(np.dot(doc_embeddings[candidate_indices], query_embedding.T))]
        else:
            selected_embeddings = doc_embeddings[selected_indices]
            similarity_to_selected = np.dot(doc_embeddings[candidate_indices], selected_embeddings.T)
            diversity_scores = np.max(similarity_to_selected, axis=1)
            relevance_scores = np.dot(doc_embeddings[candidate_indices], query_embedding.T).flatten()
            mmr_scores = (1 - diversity) * relevance_scores - diversity * diversity_scores
            selected_idx = candidate_indices[np.argmax(mmr_scores)]

        selected_indices.append(selected_idx)
        candidate_indices.remove(selected_idx)

    return selected_indices

def mmr_sampling(dataframe, sampling_dict, embedding_model='all-MiniLM-L6-v2', batch_size=64, diversity=0.7, seed=42):
    set_seed(seed)

    print("Loading SentenceTransformer model on GPU...")
    model = SentenceTransformer(embedding_model, device="cuda:0")

    sampled_rows = []

    for dataset_name, sample_count in tqdm(sampling_dict.items(), desc="Processing datasets"):
        subset = dataframe[(dataframe['dataset'] == dataset_name) & (dataframe['split_data'] == 'test')].copy()
        num_rows = len(subset)
        if num_rows == 0:
            print(f"No data found for dataset: {dataset_name}")
            continue

        print(f"Generating embeddings for dataset: {dataset_name} ({num_rows} rows)")
        start_time = time.time()

        embeddings = model.encode(
            subset['for_sampling'].astype(str).tolist(),
            batch_size=batch_size,
            convert_to_numpy=True,
            show_progress_bar=True
        )

        elapsed_time = time.time() - start_time
        print(f"Embedding generation took {elapsed_time:.2f} seconds for {num_rows} rows")

        query_embedding = np.mean(embeddings, axis=0)

        print(f"Applying MMR for dataset: {dataset_name}")
        selected_indices = mmr(query_embedding, embeddings, diversity, sample_count)
        sampled_subset = subset.iloc[selected_indices]
        sampled_rows.append(sampled_subset)

    return pd.concat(sampled_rows, ignore_index=True)

sampling_config = {
    'medqa': 100,
    'medmcqa': 100,
    'pubmedqa': 100, # 150 
    'bionli': 76, # 225
    'chemdner': 65,
    'ddi': 40,
    'meddialog': 100, 
    'pubmed': 100,
}

set_seed(42)

# 샘플링 실행
sampled_test = mmr_sampling(diabetes3, sampling_config, batch_size=4096, diversity=0.7, seed=42)

# 결과 확인
print(sampled_test.head())
print(f"총 샘플링된 행 수: {len(sampled_test)}")

# 결과 저장
sampled_test.to_csv("final_combined_test_sample.csv")


Loading SentenceTransformer model on GPU...


Processing datasets:   0%|          | 0/8 [00:00<?, ?it/s]

Generating embeddings for dataset: medqa (119 rows)


Processing datasets:  12%|█▎        | 1/8 [00:00<00:01,  5.79it/s]

Embedding generation took 0.14 seconds for 119 rows
Applying MMR for dataset: medqa
Generating embeddings for dataset: medmcqa (145 rows)


...

Embedding generation took 0.80 seconds for 999 rows
Applying MMR for dataset: meddialog
Generating embeddings for dataset: pubmed (1703 rows)


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.50s/it]


Embedding generation took 1.94 seconds for 1703 rows
Applying MMR for dataset: pubmed


Processing datasets: 100%|██████████| 8/8 [00:04<00:00,  1.79it/s]

  dataset split_data                                           features  \
0   medqa       test  {'question': 'A 60-year-old woman presents to ...   
1   medqa       test  {'question': 'A 72-year-old male presents to a...   
2   medqa       test  {'question': 'A 65-year-old female patient wit...   
3   medqa       test  {'question': 'A randomized controlled trial wa...   
4   medqa       test  {'question': 'A 62-year-old male presents to h...   

                                               input  output  \
0  A 60-year-old woman presents to the emergency ...       1   
1  A 72-year-old male presents to a cardiac surge...       1   
2  A 65-year-old female patient with a past medic...       1   
3  A randomized controlled trial was initiated to...       1   
4  A 62-year-old male presents to his primary car...       1   

                                        for_sampling  
0  {'input': 'A 60-year-old woman presents to the...  
1  {'input': 'A 72-year-old male presents to a ca...  

In [11]:
import pandas as pd

final_combined_train_sample = pd.read_csv("/data/jaesung/llm_for_diabetes/src/data/data1_diabetes/final_combined_train_sample.csv")
final_combined_test_sample = pd.read_csv("/data/jaesung/llm_for_diabetes/src/data/data1_diabetes/final_combined_test_sample.csv")

In [12]:
final_combined_train_sample['dataset'].value_counts()

dataset
medqa        500
medmcqa      500
pubmedqa     500
bionli       500
pubmed       500
meddialog    500
chemdner     416
ddi          300
Name: count, dtype: int64

In [13]:
# 중복 제거 후 겹치는 개수 확인
train_unique = final_combined_train_sample.drop_duplicates(subset='features')
test_unique = final_combined_test_sample.drop_duplicates(subset='features')

# train/test 겹치는 개수 계산
overlapping_count = len(pd.merge(train_unique, test_unique, on='features'))
print(f"Train/Test 겹치는 feature 개수: {overlapping_count}")


Train/Test 겹치는 feature 개수: 0


In [14]:
import numpy as np

final_combined_train_sample['task'] = None
# medqa
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'medqa', 'task'] = 'qa1'
# medmcqa
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'medmcqa', 'task'] = 'qa2'
# pubmedqa
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'pubmedqa', 'task'] = 'qa3'
# bionli
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'bionli', 'task'] = 'nli'
# chemdner
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'chemdner', 'task'] = 'ie'
# ddi
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'ddi', 'task'] = 're'
# meddialog
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'meddialog', 'task'] = 'generation'
# pubmed
final_combined_train_sample.loc[final_combined_train_sample['dataset'] == 'pubmed', 'task'] = 'summarization'

In [15]:
import numpy as np

final_combined_test_sample['task'] = None
# medqa
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'medqa', 'task'] = 'qa1'
# medmcqa
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'medmcqa', 'task'] = 'qa2'
# pubmedqa
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'pubmedqa', 'task'] = 'qa3'
# bionli
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'bionli', 'task'] = 'nli'
# chemdner
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'chemdner', 'task'] = 'ie'
# ddi
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'ddi', 'task'] = 're'
# meddialog
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'meddialog', 'task'] = 'generation'
# pubmed
final_combined_test_sample.loc[final_combined_test_sample['dataset'] == 'pubmed', 'task'] = 'summarization'

In [17]:
import ast
import random

def parse_features(row):
    try:
        return ast.literal_eval(row)
    except (ValueError, SyntaxError):
        return {}

    
def generate_input_output(row):
    instruction_value = ""; input_value = ""; output_value = ""

    task = row['task']
    features = parse_features(row['features'])


    if row['dataset']=='medqa' and task=='qa1':
        question = features.get('question', None)
        options = features.get('options', None)
        answer = features.get('answer', None)
        answer_idx = features.get('answer_idx', None)
        instruction_value =  "You are given a multiple-choice medical question with four possible answers. Your task is to identify and select the single most appropriate answer. based on your medical knowledge. Provide your response as a single letter: A, B, C, or D."
        input_value = (
            f"{question} Please select one of the following: A) {options['A']}, B) {options['B']}, C) {options['C']}, D) {options['D']}."
        )
        output_value = (
            f"{answer_idx}"
        )
    elif row['dataset']=='medmcqa' and task=='qa2': 
        question = features.get('question', None)
        options = {
            'A': features.get('opa', '').strip(),
            'B': features.get('opb', '').strip(),
            'C': features.get('opc', '').strip(),
            'D': features.get('opd', '').strip(),
        }
        answer_num = features.get('cop', None)
        answer_idx = 'A' if answer_num == 0 else 'B' if answer_num == 1 else 'C' if answer_num == 2 else 'D'
        answer = features.get('opa', '').strip() if answer_num == 0 else features.get('opb', '').strip() if answer_num == 1 else features.get('opc', '').strip() if answer_num == 2 else features.get('opd', '').strip()
        instruction_value =  "You are given a multiple-choice medical question with four possible answers. Your task is to identify and select the single most appropriate answer. based on your medical knowledge. Provide your response as a single letter: A, B, C, or D."
        input_value = (
            f"{question} Please select one of the following: A) {options['A']}, B) {options['B']}, C) {options['C']}, D) {options['D']}."
        )
        output_value = (
            f"{answer_idx}"
        )
    elif row['dataset']=='pubmedqa' and task=='qa3':
        question = features.get('QUESTION', '').strip()
        context = " ".join(features.get('CONTEXTS'))
        # answer = features.get('LONG_ANSWER', '').strip() 
        answer = features.get('final_decision', '').strip()
        instruction_value = "Choose the correct anser (Yes, No, or Maybe) for the given question based on the proviced context."
        input_value = (
            f"Question: {question} "
            f"Context: {context}"
        )
        output_value = f"{answer}"
    elif row['dataset']=='bionli' and task=='nli':
        query = features.get('query', '') 
        instruction_value = "Analyze the logical relationship between the given premise and hypothesis. Your task is to classify this relationship into one of the following categories: Entailment: Select this when the hypothesis logically follows from the premise. In other words, if the premise is true, the hypothesis must also be true Contradiction: Select this when the hypothesis directly contradicts the premise. This means that if the premise is true, the hypothesis must be false. Please choose only one option: 'entailment' or 'contradiction'."

        input_value = query.split("INPUT: ")[1].split("[HYP]")[0].strip() if "INPUT: " in query else None
        output_value = features.get('answer', None)
    
    elif row['dataset']=='chemdner' and task=='ie':
        instruction_value = "Extract chemical entities from the given text, ensuring they belong to one of these categories: Chemical Compounds (e.g., glucose, ATP, cholesterol), Drugs & Pharmaceuticals (e.g., metformin, aspirin, paclitaxel), Metabolites & Bioactive Molecules (e.g., serotonin, nitric oxide, kynurenine) ,Vitamins & Minerals (e.g., vitamin D, zinc, calcium), Toxic & Environmental Chemicals (e.g., arsenic, BPA, dioxin). Guidelines: Extract only named entities from these categories. Exclude general chemical properties (e.g., acidity, solubility) and chemical processes (e.g., oxidation, hydrolysis). Keep entity names as they appear in the text. Return only the extracted names."

        input_value = features.get('text', None)
        answer = features.get('entities', None)
        if isinstance(answer, list) and len(answer) > 0:
            output_value = ', '.join(answer)
        else:
            output_value = ''
    elif row['dataset']== 'ddi' and task=='re':
        instruction_value = "Analyze the sentence with two drugs labeled as @DRUG_A$ and @DRUG_B$. Extract the interaction between @DRUG_A$ and @DRUG_B$ from the input sentence by selecting only one of the following options: 'DDI-effect', 'DDI-mechanism', 'DDI-advise', 'DDI-false', and 'DDI-int'. 'DDI-effect': Choose this when the interaction describes an effect or a pharmacodynamic mechanism. 'DDI-mechanism': Choose this for interactions explained by pharmacokinetic mechanisms. 'DDI-advise': Choose this when the sentence provides a recommendation or advice about the drug interaction. 'DDI-false': Choose this if there is no actual drug-drug interaction in the sentence. 'DDI-int': Choose this when a drug-drug interaction is mentioned without additional detail."
        input_value = features.get('conversations', [])[0].get('value', None).split("INPUT:")[1].split("OUTPUT:")[0].strip()
        output_value = features.get('conversations', [])[1].get('value', None)
    elif row['dataset']=='meddialog' and task=='generation':
        # instruction_value = features.get('tgt', None)
        instruction_value = "As a medical AI assistant, generate a professional and medically accurate response to the given patient query. Ensure that your response aligns with medical guidelines and provides clear, practical advice. Avoid assumptions and base your answer solely on the provided information."
        input_value = features.get('src', None).split("Doctor:")[0].replace("Patient:", "").strip()
        output_value = features.get('src', None).split("Doctor:")[1].strip()
    elif row['dataset']=='pubmed' and task=='summarization':
        instruction_value = "Summarize the following doctor-patient conversation into a structured clinical note."
        article = (features.get('conversation') or '').strip()  
        abstract = (features.get('data')or '').strip() 
        input_value = f"{article}"
        output_value = abstract
    return instruction_value, input_value, output_value

final_combined_train_sample[['instruction', 'input', 'output']] = final_combined_train_sample.apply(
    lambda row: pd.Series(generate_input_output(row)), axis=1
)

final_combined_test_sample[['instruction', 'input', 'output']] = final_combined_test_sample.apply(
    lambda row: pd.Series(generate_input_output(row)), axis=1
)

In [18]:
final_combined_train_sample[final_combined_train_sample['task']=='qa3']['output'].value_counts()

output
yes    461
no      39
Name: count, dtype: int64

In [19]:
final_combined_train_sample[['task', 'dataset']].value_counts()

task           dataset  
generation     meddialog    500
nli            bionli       500
qa1            medqa        500
qa2            medmcqa      500
qa3            pubmedqa     500
summarization  pubmed       500
ie             chemdner     416
re             ddi          300
Name: count, dtype: int64

In [9]:
# import pandas as pd

# # 균형 조정을 위한 함수 정의
# def balance_classes(df, task_name, target_ratio=3):
#     """
#     특정 task에서 'yes'와 'no'의 비율을 target_ratio:1로 맞춤
#     """
#     # 해당 task만 필터링
#     task_df = df[df['task'] == task_name]
#     other_tasks_df = df[df['task'] != task_name]  # 다른 task 데이터는 그대로 유지

#     # 'yes'와 'no' 분리
#     yes_df = task_df[task_df['output'] == 'yes']
#     no_df = task_df[task_df['output'] == 'no']

#     # 목표 yes 샘플 개수 설정 (no의 target_ratio 배)
#     target_yes_count = len(no_df) * target_ratio

#     # 'yes' 데이터 downsampling
#     yes_sampled = yes_df.sample(n=target_yes_count, random_state=42)

#     # 균형을 맞춘 task 데이터셋 생성
#     balanced_task_df = pd.concat([yes_sampled, no_df])

#     # 다른 task 데이터와 다시 합쳐서 반환
#     balanced_df = pd.concat([balanced_task_df, other_tasks_df])

#     return balanced_df

# # Train 데이터 균형 조정 (qa_objective_3만 수정)
# final_combined_train_sample = balance_classes(final_combined_train_sample, 'qa_objective_3')

# # Test 데이터 균형 조정 (qa_objective_3만 수정)
# final_combined_test_sample = balance_classes(final_combined_test_sample, 'qa_objective_3')

In [10]:
# final_combined_train_sample[final_combined_train_sample['task']=='qa_objective_3']['output'].value_counts()

In [11]:
# final_combined_train_sample[['task','dataset']].value_counts()

In [12]:
# import pandas as pd
# import random

# # IE 데이터 필터링 (Train & Test)
# ie_train_df = final_combined_train_sample[final_combined_train_sample['task'] == 'ie'].copy()
# ie_test_df = final_combined_test_sample[final_combined_test_sample['task'] == 'ie'].copy()

# # Instruction 증강 함수
# def augment_instruction(instruction, task):
#     """
#     IE 태스크에 대해 다양한 형태의 Instruction을 생성하여 증강하는 함수.
#     """
#     instruction_variations = {
#         "ie": [
#             "Extract all chemical entities mentioned in the given text. The extracted entities must fall into one of the following categories: Chemical Compounds: Substances with a defined molecular structure, including organic and inorganic compounds. Examples: glucose, cholesterol, ATP, quercetin. Drugs & Pharmaceuticals: Medications and bioactive compounds used for treatment, diagnosis, or prevention of diseases. Examples: metformin, aspirin, atorvastatin, warfarin, paclitaxel. Metabolites & Bioactive Molecules: Naturally occurring molecules that play a role in biological processes. Examples: serotonin, cortisol, nitric oxide (NO), superoxide, kynurenine. Vitamins & Minerals: Essential nutrients required for biological functions. Examples: vitamin D, vitamin C, zinc, calcium, magnesium. Toxic & Environmental Chemicals: Pollutants and hazardous chemicals found in the environment. Examples: arsenic, polychlorinated biphenyls (PCB), dioxin (TCDD), bisphenol A (BPA). Exclude general chemical properties or reaction mechanisms. Maintain the original entity names without abbreviation or alteration.",

#             "Identify all chemical substances present in the given text, ensuring they belong to one of the predefined categories: Chemical Compounds: Defined molecular substances like glucose, cholesterol, and ATP. Drugs & Pharmaceuticals: Medicinal substances such as aspirin, atorvastatin, and paclitaxel. Metabolites & Bioactive Molecules: Naturally occurring biological compounds, including serotonin and nitric oxide. Vitamins & Minerals: Essential nutrients like vitamin D, zinc, and calcium. Toxic & Environmental Chemicals: Pollutants such as arsenic, dioxin, and bisphenol A. Exclude general chemical descriptions and processes. Extract only explicitly named entities.",

#             "Detect and list all named chemical entities in the text, categorizing them into one of the following: Chemical Compounds: Molecularly defined substances (e.g., ATP, quercetin). Drugs & Pharmaceuticals: Medications and active compounds (e.g., metformin, warfarin). Metabolites & Bioactive Molecules: Biological molecules affecting physiological functions (e.g., cortisol, superoxide). Vitamins & Minerals: Essential nutrients required for health (e.g., vitamin C, magnesium). Toxic & Environmental Chemicals: Hazardous environmental substances (e.g., PCB, BPA, dioxin). Ensure that extracted entities match these categories without modification. Do not extract chemical properties or reaction descriptions.",

#             "Extract chemical entities explicitly named in the text, making sure they fit into one of the specified categories: Chemical Compounds: Organic and inorganic molecules like ATP and glucose. Drugs & Pharmaceuticals: Medicinal and bioactive compounds such as aspirin and atorvastatin. Metabolites & Bioactive Molecules: Naturally occurring biochemical substances like nitric oxide and serotonin. Vitamins & Minerals: Essential micronutrients, including vitamin D, calcium, and zinc. Toxic & Environmental Chemicals: Harmful environmental compounds such as dioxin and bisphenol A. Exclude general chemical properties or reaction mechanisms. Maintain the original entity names without abbreviation or alteration."
#         ]

#     }
#     return random.choice(instruction_variations.get(task, [instruction]))  # 기본적으로 원래 instruction 포함

# # 데이터 증강 함수 (IE는 5배 증강, 나머지 컬럼 유지)
# def augment_ie_data_preserve(df):
#     augmented_data = []

#     for _, row in df.iterrows():
#         task = row['task']

#         # IE만 5배 증강
#         augmentation_factor = 4 if task == 'ie' else 1

#         for _ in range(augmentation_factor):
#             new_row = row.copy()  # 원본 행 복사하여 모든 컬럼 유지
#             new_row['instruction'] = augment_instruction(row['instruction'], task)  # Instruction만 변경
#             augmented_data.append(new_row)

#     return pd.DataFrame(augmented_data)

# # IE 데이터 증강 수행 (Train & Test)
# ie_train_augmented_df = augment_ie_data_preserve(ie_train_df)
# ie_test_augmented_df = augment_ie_data_preserve(ie_test_df)

# # 원본 데이터와 결합하여 최종 증강된 데이터 생성 (Train & Test)
# final_combined_train_sample = pd.concat([final_combined_train_sample, ie_train_augmented_df], ignore_index=True)
# final_combined_test_sample = pd.concat([final_combined_test_sample, ie_test_augmented_df], ignore_index=True)


In [22]:
final_combined_train_sample[['task','dataset']].value_counts()

task           dataset  
generation     meddialog    500
nli            bionli       500
qa1            medqa        500
qa2            medmcqa      500
qa3            pubmedqa     500
summarization  pubmed       500
ie             chemdner     416
re             ddi          300
Name: count, dtype: int64

In [23]:
final_df = pd.concat([final_combined_train_sample, final_combined_test_sample])

In [24]:
final_combined_train_sample.to_json("/data/jaesung/llm_for_diabetes/src/data/data1_diabetes/train_instruction_dataset.json", orient="columns", indent=4)
final_combined_test_sample.to_json("/data/jaesung/llm_for_diabetes/src/data/data1_diabetes/test_instruction_dataset.json", orient="columns", indent=4)

In [25]:
final_combined_train_sample.head(2)

,Unnamed: 0,dataset,split_data,features,input,output,for_sampling,task,instruction
0,0,medqa,train,{'question': 'A 52-year-old man presents to th...,A 52-year-old man presents to the emergency de...,A,{'input': 'A 52-year-old man presents to the e...,qa1,You are given a multiple-choice medical questi...
1,1,medqa,train,{'question': 'A 72-year-old woman with type 2 ...,A 72-year-old woman with type 2 diabetes melli...,C,{'input': 'A 72-year-old woman with type 2 dia...,qa1,You are given a multiple-choice medical questi...
